In [1]:
%load_ext autoreload
%autoreload 2

import os,sys,warnings
import h5py, pandas
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
np.set_printoptions(legacy='1.25')
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit,fsolve
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 24
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 6
mpl.rcParams['errorbar.capsize'] = 6
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

# mpl.rcParams.update({"axes.grid" : True})
import util as yu
yu.flag_fast=False

enss=['b','c','d','e']
enss=['b','c','d']
ens2full={'a24':'cA211.53.24','a':'cA2.09.48','b':'cB211.072.64','c':'cC211.060.80','d':'cD211.054.96','e':'cE211.044.112'}
ens2label={'a24':'A24','a':'A48','b':'B64','c':'C80','d':'D96','e':'E112'}
ens2a={'a24':0.0908,'a':0.0938,'b':0.07957,'c':0.06821,'d':0.05692,'e':0.04892} # fm
ens2N={'a24':24,'a':48,'b':64,'c':80,'d':96,'e':112}
ens2N_T={'a24':24*2,'a':48*2,'b':64*2,'c':80*2,'d':96*2,'e':112*2}

# ens2mN={'a':931}
# ens2mpiC={'a':131}
# ens2mpi0={'a':111}

hbarc = 1/197.3
ens2aInv={ens:1/(ens2a[ens]*hbarc) for ens in enss} # MeV

baseFigPath='fig/analysis_A20/'

def find_t_cloest(ens,t):
    return round(t/ens2a[ens])

path_basedata='/p/project1/ngff/li47/code/projectData/02_discNJN_1D/analysis_A20.h5'

noNumbersOnLegend=True
noNumbersOnBar=False

In [2]:
# RCs
ens2Njk={}
for ens in enss:
    path=path_basedata
    with h5py.File(path) as f:        
        ens2Njk[ens]=len(f[f'{ens}/mom0/N'])

path='data_aux/RCs.pkl'
with open(path,'rb') as f:
    ens2RCs_me=pickle.load(f)
ens2RCs={ens:{} for ens in enss}
for ens in enss:
    for key in ens2RCs_me[ens]:
        if key.endswith('err'):
            continue
        ens2RCs[ens][key]=yu.jackknife_pseudo([ens2RCs_me[ens][key]],np.array([[ens2RCs_me[ens][f'{key}_err']**2+1e-10]]),ens2Njk[ens])[:,0]
use='C'
if use == 'Y':
    # my results
    path='data_aux/dat_ignore/analysis_A20.pkl'
    with open(path,'rb') as f:
        ens2bare=pickle.load(f)
    stouts=range(0,40+1,2)
    stout_chosen=10
    for ens in ens2bare.keys():
        for j in ['j+','j-']:
            ens2bare[ens][f'{j};conn']=ens2bare[ens][f'mom0_{j};conn']
        for j in ['j+','js','jc']:
            ens2bare[ens][f'{j};disc']=ens2bare[ens][f'mom1_{j};disc_sum']
        for stout in stouts:
            for j in ['jg']:
                ens2bare[ens][f'{j};disc;{stout}']=ens2bare[ens][f'mom0_{j};stout{stout};disc']
    munu_jqC='(mu=nu)'; munu_jqD='(mu!=nu)'; munu_jg='(mu=nu)'
elif use == 'C':
    # Chrisitian's results
    path='data_aux/dat_ignore/Christian_analysis_A20.pkl'
    stouts=[20]
    stout_chosen=20
    with open(path,'rb') as f:
        ens2bare=pickle.load(f)
    for ens in ens2bare.keys():
        ens2bare[ens]['jg;disc;20']=ens2bare[ens]['jg;disc']
    munu_jqC='(mu=nu)'; munu_jqD='(mu=nu)'; munu_jg='(mu=nu)' # Christian has rescaled the bare value to be multiplied by mu=nu RCs
    

In [3]:
# superjackknife

fla2iso={
    'u':[(1/4,'q'),(1/2,'v1'),(1/6,'v2'),(1/12,'v3')],
    'd':[(1/4,'q'),(-1/2,'v1'),(1/6,'v2'),(1/12,'v3')],
    's':[(1/4,'q'),(-1/3,'v2'),(1/12,'v3')],
    'c':[(1/4,'q'),(-1/4,'v3')]
}
fla2iso_conn={
    'u':[(1/2,'q'),(1/2,'v1')],
    'd':[(1/2,'q'),(-1/2,'v1')],
}

ens2rb={ens:{} for ens in enss}
ens2rb['a=#_MA']={}; ens2rb['a=#_const']={}; ens2rb['a=#_linear']={}

for stout in stouts:
    for ens in enss:
        dic_rb=ens2rb[ens]
        dic_b=ens2bare[ens]
        dic_Z=ens2RCs[ens]
        
        dic_b[f'jq;disc']=dic_b[f'j+;disc']+dic_b[f'js;disc']+dic_b[f'jc;disc']

        dic_rb[f'jq;{stout}']=dic_Z[f'Zqq^s{munu_jqC}']*dic_b[f'j+;conn'] + dic_Z[f'Zqq^s{munu_jqD}']*dic_b[f'jq;disc'] + \
            dic_Z[f'Zqg{munu_jg}']*dic_b[f'jg;disc;{stout}']
        dic_rb[f'jg;{stout}']=dic_Z[f'Zgq{munu_jqC}']*dic_b[f'j+;conn'] + dic_Z[f'Zgq{munu_jqD}']*dic_b[f'jq;disc'] + \
            dic_Z[f'Zgg^{stout}{munu_jg}']*dic_b[f'jg;disc;{stout}']
            
        dic_rb[f'jv1;{stout}']=dic_Z[f'Zqq{munu_jqC}']*dic_b[f'j-;conn']
        dic_rb[f'jv2;{stout}']=dic_Z[f'Zqq{munu_jqC}']*dic_b[f'j+;conn'] + dic_Z[f'Zqq{munu_jqD}']*(dic_b[f'j+;disc']-2*dic_b[f'js;disc'])
        dic_rb[f'jv3;{stout}']=dic_Z[f'Zqq{munu_jqC}']*dic_b[f'j+;conn'] + dic_Z[f'Zqq{munu_jqD}']*(dic_b[f'j+;disc']+dic_b[f'js;disc']-3*dic_b[f'jc;disc'])
            
        dic_rb[f'jq;conn;{stout}']=dic_Z[f'Zqq^s{munu_jqC}']*dic_b[f'j+;conn']
        dic_rb[f'jv1;conn;{stout}']=dic_Z[f'Zqq{munu_jqC}']*dic_b[f'j-;conn']
        dic_rb[f'jtot;conn;{stout}']=dic_rb[f'jq;conn;{stout}']

        dic_rb[f'jtot;{stout}']=dic_rb[f'jq;{stout}']+dic_rb[f'jg;{stout}']
        
        for fla in fla2iso.keys():
            dic_rb[f'j{fla};{stout}']=np.sum([factor*dic_rb[f'j{iso};{stout}'] for factor,iso in fla2iso[fla]],axis=0)
        for fla in fla2iso_conn.keys():
            dic_rb[f'j{fla};conn;{stout}']=np.sum([factor*dic_rb[f'j{iso};conn;{stout}'] for factor,iso in fla2iso_conn[fla]],axis=0)
        
    # continuum extrapolation
    a2s_plt=np.arange(0,0.011,0.001)
    for j in ['jg','jq','jv1','jv2','jv3','jq;conn','jv1;conn']:
        a2s=np.array([ens2a[ens]**2 for ens in enss])
        y_jk=yu.superjackknife([ens2rb[ens][f'{j};{stout}'][:,None] for ens in enss])
        
        fits=[]
        
        def fitfunc(pars):
            g=pars
            return g+0*a2s
        pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,y_jk,[1])
        def fitfunc_plt(pars):
            g=pars
            return g+0*a2s_plt
        pars_plt=yu.jackmap(fitfunc_plt,pars_jk)
        ens2rb['a=#_const'][f'{j};{stout}']=pars_plt
        fits.append(['const',pars_plt,chi2_jk,Ndof])
        
        def fitfunc(pars):
            g,c=pars
            return g+c*a2s
        pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,y_jk,[1,1])
        def fitfunc_plt(pars):
            g,c=pars
            return g+c*a2s_plt
        pars_plt=yu.jackmap(fitfunc_plt,pars_jk)
        ens2rb['a=#_linear'][f'{j};{stout}']=pars_plt
        fits.append(['linear',pars_plt,chi2_jk,Ndof])
        
        pars_plt,props_jk=yu.jackMA(fits)
        ens2rb['a=#_MA'][f'{j};{stout}']=pars_plt
    
    for ft in ['const','linear','MA']:
        ens2rb[f'a=#_{ft}'][f'jtot;{stout}']=ens2rb[f'a=#_{ft}'][f'jq;{stout}']+ens2rb[f'a=#_{ft}'][f'jg;{stout}']
        ens2rb[f'a=#_{ft}'][f'jtot;conn;{stout}']=ens2rb[f'a=#_{ft}'][f'jq;conn;{stout}']
    
        for fla in fla2iso.keys():
            ens2rb[f'a=#_{ft}'][f'j{fla};{stout}']=np.sum([factor*ens2rb[f'a=#_{ft}'][f'j{iso};{stout}'] for factor,iso in fla2iso[fla]],axis=0)
        for fla in fla2iso_conn.keys():
            ens2rb[f'a=#_{ft}'][f'j{fla};conn;{stout}']=np.sum([factor*ens2rb[f'a=#_{ft}'][f'j{iso};conn;{stout}'] for factor,iso in fla2iso_conn[fla]],axis=0)

ens2rb_A20=ens2rb.copy()

In [77]:
# for j in ['jq','jg']:
#     t=ens2rb_A20['b'][f'{j};20']
#     mean,err=yu.jackme(t)
#     print(j,yu.un2str(mean,err))
# print()
# for j in ['j+;conn','jq;disc','j+;disc','js;disc','jc;disc','jg;disc;20']:
#     t=ens2bare['b'][j]
#     mean,err=yu.jackme(t)
#     print(j,yu.un2str(mean,err))

In [11]:
path='data_aux/dat_ignore/analysis_A20_B20_from_analysis3_unequal_0sink.pkl'
path='data_aux/dat_ignore/analysis_A20_B20_from_analysis3_removeQ2.pkl'
with open(path,'rb') as f:
    ens2rb_B20=pickle.load(f)

In [5]:
if False:
    # js=['jtot','jg','jq']
    js=['jg']

    fig, axs = yu.getFigAxs(len(js),1,Lrow=4,Lcol=8)

    stouts_plt=range(0,31)

    for ij,j in enumerate(js):
        ax=axs[ij,0]
        ax.set_ylim({'jtot':[0.5,1.5],'jg':[0,1],'jq':[0,1]}[j])
        ax.set_ylabel(r'$\langle \mathrm{x} \rangle _g$')
        ax.set_xlabel(r'$n_s$')
        if j=='jtot':
            ax.axhline(1,ls='--')
        
        colors=['r','g','b']; fmts=['s','o','d']
        for stout in stouts_plt:
            if stout%2!=0:
                continue
            for iens,ens in enumerate(enss):
                plt_x=stout+0.2*iens; plt_y,plt_yerr=yu.jackme(ens2rb[ens][f'{j};{stout}'])
                ax.errorbar(plt_x,plt_y,plt_yerr,color=colors[iens],fmt=fmts[iens],label=ens2label[ens] if stout==stouts[0] else None)

            # plt_x=stout-0.2; plt_y,plt_yerr=yu.jackme(ens2rb['a=#_MA'][f'{j};{stout}'][:,0])
            # ax.errorbar(plt_x,plt_y,plt_yerr,color='orange',label='c.l._MA' if stout==stouts[0] else None)
                
            # if stout==10:
            #     print(j,'MA',plt_y,plt_yerr)
            
            # plt_x=stout-0.4; plt_y,plt_yerr=yu.jackme(ens2rb['a=#_const'][f'{j};{stout}'][:,0])
            # ax.errorbar(plt_x,plt_y,plt_yerr,color='purple',label='c.l._const' if stout==stouts[0] else None)
            # if stout==10:
            #     print(j,'const',plt_y,plt_yerr)
            # plt_x=stout-0.6; plt_y,plt_yerr=yu.jackme(ens2rb['a=#_linear'][f'{j};{stout}'][:,0])
            # ax.errorbar(plt_x,plt_y,plt_yerr,color='brown',label='c.l._linear' if stout==stouts[0] else None)
            # if stout==10:
            #     print(j,'linear',plt_y,plt_yerr)

        # if j=='jtot':
            ax.legend(fontsize=10)
    print()
    plt.tight_layout()
    os.makedirs(f'{baseFigPath}/final1',exist_ok=True)
    # plt.savefig(f'{baseFigPath}/final1/stoutDependence_x.pdf',bbox_inches="tight")
    plt.close()

# avg x

In [5]:
def get(ens,j):
    return ens2rb_A20[ens][j]

temp={}

stout=stout_chosen

j2color={'jq':'purple','jg':'cyan','jtot':'grey','ju':'r','jd':'g','js':'b','jc':'orange'}
j2label={'jq':'q','jg':'g','jtot':'N','ju':'u','jd':'d','js':'s','jc':'c'}
j2fmt={'jq':'d','jg':'s','jtot':'o','ju':'^','jd':'v','js':'<','jc':'>'}

fig, axs = yu.getFigAxs(2,1,Lrow=4,Lcol=6,sharex=True)
ax=axs[0,0]
ax.set_xlim([0,0.01])
ax.set_xticks([0,0.003,0.006,0.009])
ax.set_ylim([0.25,1.35])
ax.set_yticks([0.4,0.6,0.8,1.0,1.2])
ax.set_ylabel(r'$\langle \mathrm{x} \rangle_{q,g}$')
ax.axhline(1,color='black',ls='--',marker='')
# ax.axvline(0,color='black',ls='dotted',marker='')
js=['jtot','jq','jg']
for ij,j in enumerate(js):
    color=j2color[j]
    mean,err=yu.jackme(get('a=#_MA',f'{j};{stout}'))
    label=rf'$\langle x\rangle _{{{j2label[j]}}}=$'+yu.un2str(mean[0],err[0],forceResult=1)
    if noNumbersOnLegend:
        label=rf'$\langle x\rangle _{{{j2label[j]}}}$'
    for iens,ens in enumerate(enss):
        plt_x=ens2a[ens]**2+(ij-len(js)/2)*5e-5; plt_y,plt_yerr=yu.jackme(get(ens,f'{j};{stout}'))
        temp[f'{j}_{ens}']=yu.un2str(plt_y,plt_yerr)
        ax.errorbar(plt_x,plt_y,plt_yerr,color=color,fmt=j2fmt[j],label=label if iens==0 else None)

    mean,err=yu.jackme(get('a=#_MA',f'{j};{stout}'))
    x=a2s_plt; ymin=mean-err; ymax=mean+err
    temp[f'{j}_0']=yu.un2str(mean[0],err[0])
    ax.plot(x,mean,color=color,linestyle='--',marker='')
    ax.fill_between(x, ymin, ymax, color=color, alpha=0.1)
ax.legend(fontsize=12)

ax=axs[1,0]
ax.set_xlabel(r'$a^2$ [fm$^2$]')
ax.set_ylim([-0.1,0.5])
ax.set_ylabel(r'$\langle \mathrm{x}\rangle_{q}$')
ax.axhline(0,color='black',ls='--',marker='')
js=['ju','jd','js','jc']
for ij,j in enumerate(js):
    color=j2color[j]
    mean,err=yu.jackme(get('a=#_MA',f'{j};{stout}'))
    label=rf'$\langle x\rangle _{{{j2label[j]}}}=$'+yu.un2str(mean[0],err[0],forceResult=1)
    if noNumbersOnLegend:
        label=rf'$\langle x\rangle _{{{j2label[j]}}}$'
    for iens,ens in enumerate(enss):
        plt_x=ens2a[ens]**2+(ij-len(js)/2)*5e-5; plt_y,plt_yerr=yu.jackme(get(ens,f'{j};{stout}'))
        temp[f'{j}_{ens}']=yu.un2str(plt_y,plt_yerr)
        ax.errorbar(plt_x,plt_y,plt_yerr,color=color,fmt=j2fmt[j],label=label if iens==0 else None)

    mean,err=yu.jackme(get('a=#_MA',f'{j};{stout}'))
    x=a2s_plt; ymin=mean-err; ymax=mean+err
    temp[f'{j}_0']=yu.un2str(mean[0],err[0],forceResult=1)
    ax.plot(x,mean,color=color,linestyle='--',marker='')
    ax.fill_between(x, ymin, ymax, color=color, alpha=0.1)
ax.legend(fontsize=12,ncol=2)

plt.tight_layout()
plt.savefig(f'{baseFigPath}/final1/a2Dependence_x.pdf',bbox_inches="tight")
plt.close()

for j in ['ju','jd','js','jc','jq','jg','jtot']:
    print(' & '.join([f'$\\braket{{x}}_{{{j[1:]}}}$',temp[f'{j}_b'],temp[f'{j}_c'],temp[f'{j}_d'],temp[f'{j}_0']]),'\\\\')

$\braket{x}_{u}$ & 0.335(17) & 0.305(18) & 0.318(24) & 0.293(21) \\
$\braket{x}_{d}$ & 0.156(14) & 0.148(16) & 0.161(20) & 0.149(15) \\
$\braket{x}_{s}$ & 0.010(12) & 0.014(14) & 0.031(19) & 0.035(15) \\
$\braket{x}_{c}$ & -0.0219(95) & -0.017(12) & 2(17)e-3 & 0.005(12) \\
$\braket{x}_{q}$ & 0.478(46) & 0.450(57) & 0.513(78) & 0.482(51) \\
$\braket{x}_{g}$ & 0.449(26) & 0.494(34) & 0.460(44) & 0.478(31) \\
$\braket{x}_{tot}$ & 0.927(57) & 0.945(70) & 0.972(98) & 0.960(64) \\


In [6]:
stout=stout_chosen

def get(j,stout,part):
    if part=='full':
        return ens2rb['a=#_MA'][f'{j};{stout}'][:,0]
    if part=='conn':
        if j not in ['ju','jd','jq','jtot']:
            return ens2rb['a=#_MA'][f'{j};{stout}'][:,0]*0
        return ens2rb['a=#_MA'][f'{j};conn;{stout}'][:,0]

fig, axs = yu.getFigAxs(1,1,Lrow=6,Lcol=8)

js = ['ju','jd','js','jc','jq','jg','jtot']
# labels = [r'$u^+$', r'$d^+$', r'$s^+$', r'$c^+$', r'$\sum_{q^+ = u,d,s,c}$', r'$g$', 'Total']
labels = [r'$u$', r'$d$', r'$s$', r'$c$', r'$q$', r'$g$', 'Total']
colors = ['r','g','b','orange','purple','cyan','grey']

ax=axs[0,0]
ax.axhline(1,color='black',ls='--',marker='')
ax.axhline(0,color='black',ls='--',marker='')
ax.set_ylabel(r'$\langle \mathrm{x}\rangle_{q,g}$')

x = np.arange(len(labels))
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_xlim([x[0]-3/4,x[-1]+3/4])

mes_conn=[yu.jackme(get(j,stout,'conn')) for j in js]
mes=[yu.jackme(get(j,stout,'full')) for j in js]

factor=1
    
ax.bar(x, [ele[0] for ele in mes_conn], capsize=5, color=colors, alpha=0.8, width=0.3, edgecolor='grey')
bars=ax.bar(x, [ele[0] for ele in mes], yerr=[ele[1] for ele in mes], capsize=5, color=colors, alpha=0.2, width=0.5, edgecolor='grey')

func=lambda res:'{:.1f}({:.1f})%'.format(res[0]*100/factor,res[1]*100/factor)
percentages=[func(ele) for ele in mes]

if not noNumbersOnBar:
    for i, (bar, pct) in enumerate(zip(bars, percentages)):
        height = bar.get_height()
        height = 0.01 if height < 0.2 else height - 0.2
        ax.text(bar.get_x() - 0.1, height, pct,
                ha='center', va='bottom', fontsize=10, rotation=90)
        
plt.tight_layout()
plt.savefig(f'{baseFigPath}/final1/percentages_x.pdf',bbox_inches="tight")
plt.close()

# J

In [12]:
def get(ens,j):
    return (ens2rb_A20[ens][j]+ens2rb_B20[ens][j])/2

stout=stout_chosen

j2color={'jq':'purple','jg':'cyan','jtot':'grey','ju':'r','jd':'g','js':'b','jc':'orange'}
j2label={'jq':'q','jg':'g','jtot':'N','ju':'u','jd':'d','js':'s','jc':'c'}
j2fmt={'jq':'d','jg':'s','jtot':'o','ju':'^','jd':'v','js':'<','jc':'>'}

fig, axs = yu.getFigAxs(2,1,Lrow=4,Lcol=6,sharex=True)
ax=axs[0,0]
ax.set_xlim([0,0.01])
ax.set_xticks([0,0.003,0.006,0.009])
ax.set_ylim([0.05,0.75])
ax.set_yticks([0.1,0.3,0.5,0.7])
ax.set_ylabel(r'$J_{q,g}$')
ax.axhline(1/2,color='black',ls='--',marker='')
# ax.axvline(0,color='black',ls='dotted',marker='')
js=['jtot','jq','jg']
for ij,j in enumerate(js):
    color=j2color[j]
    mean,err=yu.jackme(get('a=#_MA',f'{j};{stout}'))
    label=rf'$J_{{{j2label[j]}}}=$'+yu.un2str(mean[0],err[0],forceResult=1)
    if noNumbersOnLegend:
        label=rf'$J_{{{j2label[j]}}}$'
    for iens,ens in enumerate(enss):
        plt_x=ens2a[ens]**2+(ij-len(js)/2)*5e-5; plt_y,plt_yerr=yu.jackme(get(ens,f'{j};{stout}'))
        ax.errorbar(plt_x,plt_y,plt_yerr,color=color,fmt=j2fmt[j],label=label if iens==0 else None)

    mean,err=yu.jackme(get('a=#_MA',f'{j};{stout}'))
    x=a2s_plt; ymin=mean-err; ymax=mean+err
    ax.plot(x,mean,color=color,linestyle='--',marker='')
    ax.fill_between(x, ymin, ymax, color=color, alpha=0.1)
ax.legend(fontsize=12)

ax=axs[1,0]
ax.set_xlabel(r'$a^2$ [fm$^2$]')
ax.set_ylim([-0.05,0.35])
ax.set_yticks([0,0.1,0.2,0.3])
ax.set_ylabel(r'$J_q$')
ax.axhline(0,color='black',ls='--',marker='')
js=['ju','jd','js','jc']
for ij,j in enumerate(js):
    color=j2color[j]
    mean,err=yu.jackme(get('a=#_MA',f'{j};{stout}'))
    label=rf'$J _{{{j2label[j]}}}=$'+yu.un2str(mean[0],err[0],forceResult=1)
    if noNumbersOnLegend:
        label=rf'$J_{{{j2label[j]}}}$'
    for iens,ens in enumerate(enss):
        plt_x=ens2a[ens]**2+(ij-len(js)/2)*5e-5; plt_y,plt_yerr=yu.jackme(get(ens,f'{j};{stout}'))
        ax.errorbar(plt_x,plt_y,plt_yerr,color=color,fmt=j2fmt[j],label=label if iens==0 else None)

    mean,err=yu.jackme(get('a=#_MA',f'{j};{stout}'))
    x=a2s_plt; ymin=mean-err; ymax=mean+err
    ax.plot(x,mean,color=color,linestyle='--',marker='')
    ax.fill_between(x, ymin, ymax, color=color, alpha=0.1)
ax.legend(fontsize=12,ncol=2)

plt.tight_layout()
plt.savefig(f'{baseFigPath}/final1/a2Dependence_J.pdf',bbox_inches="tight")
plt.close()

In [8]:
stout=stout_chosen

def get0(ens,j):
    return (ens2rb_A20[ens][j]+ens2rb_B20[ens][j])/2

def get(j,stout,part):
    if part=='full':
        return get0('a=#_MA',f'{j};{stout}')[:,0]
    if part=='conn':
        if j not in ['ju','jd','jq','jtot']:
            return get0('a=#_MA',f'{j};{stout}')[:,0]*0
        return get0('a=#_MA',f'{j};conn;{stout}')[:,0]

fig, axs = yu.getFigAxs(1,1,Lrow=6,Lcol=8)

js = ['ju','jd','js','jc','jq','jg','jtot']
# labels = [r'$u^+$', r'$d^+$', r'$s^+$', r'$c^+$', r'$\sum_{q^+ = u,d,s,c}$', r'$g$', 'Total']
labels = [r'$u$', r'$d$', r'$s$', r'$c$', r'$q$', r'$g$', 'Total']
colors = ['r','g','b','orange','purple','cyan','grey']

ax=axs[0,0]
ax.axhline(1/2,color='black',ls='--',marker='')
ax.axhline(0,color='black',ls='--',marker='')
ax.set_ylabel(r'$J_{q,g}$')

x = np.arange(len(labels))
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_xlim([x[0]-3/4,x[-1]+3/4])

mes_conn=[yu.jackme(get(j,stout,'conn')) for j in js]
mes=[yu.jackme(get(j,stout,'full')) for j in js]

factor=0.5
    
ax.bar(x, [ele[0] for ele in mes_conn], capsize=5, color=colors, alpha=0.8, width=0.3, edgecolor='grey')
bars=ax.bar(x, [ele[0] for ele in mes], yerr=[ele[1] for ele in mes], capsize=5, color=colors, alpha=0.2, width=0.5, edgecolor='grey')

func=lambda res:'{:.1f}({:.1f})%'.format(res[0]*100/factor,res[1]*100/factor)
percentages=[func(ele) for ele in mes]

if not noNumbersOnBar:
    for i, (bar, pct) in enumerate(zip(bars, percentages)):
        height = bar.get_height()
        height = 0.01 if height < 0.2 else height - 0.1
        ax.text(bar.get_x() - 0.1, height, pct,
                ha='center', va='bottom', fontsize=10, rotation=90)
        
plt.tight_layout()
plt.savefig(f'{baseFigPath}/final1/percentages_J.pdf',bbox_inches="tight")
plt.close()

# $\frac{1}{2}\Delta\Sigma = \frac{1}{2} g_A$

In [26]:
ens2q2gA={
    'b':{'ju':[0.867,0.018],'jd':[-0.408,0.014],'js':[-0.0262,0.0086],'jc':[0.0003,0.0045],'jq':[0.433,0.030],'j-':[1.275,0.023],'j+;conn':[0.585,0.013]},
    'c':{'ju':[0.832,0.022],'jd':[-0.415,0.021],'js':[-0.036,0.017],'jc':[-0.012,0.012],'jq':[0.368,0.065],'j-':[1.247,0.015],'j+;conn':[0.567,0.011]},
    'd':{'ju':[0.843,0.016],'jd':[-0.415,0.015],'js':[-0.034,0.013],'jc':[0.0076,0.0094],'jq':[0.401,0.049],'j-':[1.258,0.014],'j+;conn':[0.5659,0.0072]}
}
stout=stout_chosen
ens2rb={ens:{} for ens in enss}
for ens in enss:
    for j in ['jq','ju','jd','js','jc','j+;conn','j-']:
        ens2rb[ens][f'{j};{stout}']=yu.jackknife_pseudo([ens2q2gA[ens][j][0]],np.array([[ens2q2gA[ens][j][1]**2+1e-10]]),ens2Njk[ens])[:,0]
    ens2rb[ens][f'ju;conn;{stout}']=(ens2rb[ens][f'j+;conn;{stout}']+ens2rb[ens][f'j-;{stout}'])/2
    ens2rb[ens][f'jd;conn;{stout}']=(ens2rb[ens][f'j+;conn;{stout}']-ens2rb[ens][f'j-;{stout}'])/2
    ens2rb[ens][f'jq;conn;{stout}']=ens2rb[ens][f'j+;conn;{stout}']
    
ens2rb['a=#_MA']={}; ens2rb['a=#_const']={}; ens2rb['a=#_linear']={}

a2s_plt=np.arange(0,0.011,0.001)
for j in ['jq','ju','jd','js','jc','ju;conn','jd;conn','jq;conn']:
    a2s=np.array([ens2a[ens]**2 for ens in enss])
    y_jk=yu.superjackknife([ens2rb[ens][f'{j};{stout}'][:,None] for ens in enss])
    
    fits=[]
    
    def fitfunc(pars):
        g=pars
        return g+0*a2s
    pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,y_jk,[1])
    def fitfunc_plt(pars):
        g=pars
        return g+0*a2s_plt
    pars_plt=yu.jackmap(fitfunc_plt,pars_jk)
    ens2rb['a=#_const'][f'{j};{stout}']=pars_plt
    fits.append(['const',pars_plt,chi2_jk,Ndof])
    
    def fitfunc(pars):
        g,c=pars
        return g+c*a2s
    pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,y_jk,[1,1])
    def fitfunc_plt(pars):
        g,c=pars
        return g+c*a2s_plt
    pars_plt=yu.jackmap(fitfunc_plt,pars_jk)
    ens2rb['a=#_linear'][f'{j};{stout}']=pars_plt
    fits.append(['linear',pars_plt,chi2_jk,Ndof])
    
    pars_plt,props_jk=yu.jackMA(fits)
    ens2rb['a=#_MA'][f'{j};{stout}']=pars_plt

# for ft in ['const','linear','MA']:
#     ens2rb[f'a=#_{ft}'][f'jtot;{stout}']=ens2rb[f'a=#_{ft}'][f'jq;{stout}']+ens2rb[f'a=#_{ft}'][f'jg;{stout}']
#     ens2rb[f'a=#_{ft}'][f'jtot;conn;{stout}']=ens2rb[f'a=#_{ft}'][f'jq;conn;{stout}']

#     for fla in fla2iso.keys():
#         ens2rb[f'a=#_{ft}'][f'j{fla};{stout}']=np.sum([factor*ens2rb[f'a=#_{ft}'][f'j{iso};{stout}'] for factor,iso in fla2iso[fla]],axis=0)
#     for fla in fla2iso_conn.keys():
#         ens2rb[f'a=#_{ft}'][f'j{fla};conn;{stout}']=np.sum([factor*ens2rb[f'a=#_{ft}'][f'j{iso};conn;{stout}'] for factor,iso in fla2iso_conn[fla]],axis=0)

ens2rb_gA=ens2rb.copy()

In [27]:
def get(ens,j):
    return ens2rb_gA[ens][j]/2

stout=stout_chosen

j2color={'jq':'grey','jg':'cyan','jtot':'grey','ju':'r','jd':'g','js':'b','jc':'orange'}
j2label={'jq':'N','jg':'g','jtot':'N','ju':'u','jd':'d','js':'s','jc':'c'}
j2fmt={'jq':'d','jg':'s','jtot':'o','ju':'^','jd':'v','js':'<','jc':'>'}

fig, axs = yu.getFigAxs(1,1,Lrow=4,Lcol=6,sharex=True)
ax=axs[0,0]
ax.set_xlim([0,0.01])
ax.set_xticks([0,0.003,0.006,0.009])

ax.set_xlabel(r'$a^2$ [fm$^2$]')
ax.set_ylim([-0.6,1.0])
ax.set_yticks([-0.4,0,0.4,0.8])
ax.set_ylabel(rf'$\frac{{1}}{{2}}\Delta\Sigma_q$')
# ax.axhline(0,color='black',ls='--',marker='')
# ax.axhline(0.5,color='black',ls='--',marker='')
js=['ju','jd','js','jc','jq']
for ij,j in enumerate(js):
    color=j2color[j]
    mean,err=yu.jackme(get('a=#_MA',f'{j};{stout}'))
    label=rf'$\frac{{1}}{{2}}\Delta\Sigma_{{{j2label[j]}}}=$'+yu.un2str(mean[0],err[0],forceResult=1)
    if noNumbersOnLegend:
        label=rf'$\frac{{1}}{{2}}\Delta\Sigma_{{{j2label[j]}}}$'
    for iens,ens in enumerate(enss):
        plt_x=ens2a[ens]**2+(ij-len(js)/2)*5e-5; plt_y,plt_yerr=yu.jackme(get(ens,f'{j};{stout}'))
        ax.errorbar(plt_x,plt_y,plt_yerr,color=color,fmt=j2fmt[j],label=label if iens==0 else None)

    mean,err=yu.jackme(get('a=#_MA',f'{j};{stout}'))
    x=a2s_plt; ymin=mean-err; ymax=mean+err
    ax.plot(x,mean,color=color,linestyle='--',marker='')
    ax.fill_between(x, ymin, ymax, color=color, alpha=0.1)
ax.legend(fontsize=12,ncol=3)

plt.tight_layout()
plt.savefig(f'{baseFigPath}/final1/a2Dependence_DeltaSigmaBy2.pdf',bbox_inches="tight")
plt.close()

In [52]:
stout=stout_chosen

def get0(ens,j):
    return ens2rb_gA[ens][j]/2

def get(j,stout,part):
    if part=='full':
        return get0('a=#_MA',f'{j};{stout}')[:,0]
    if part=='conn':
        if j not in ['ju','jd','jq','jtot']:
            return get0('a=#_MA',f'{j};{stout}')[:,0]*0
        return get0('a=#_MA',f'{j};conn;{stout}')[:,0]

fig, axs = yu.getFigAxs(1,1,Lrow=6,Lcol=8)

js = ['ju','jd','js','jc','jq']
# labels = [r'$u^+$', r'$d^+$', r'$s^+$', r'$c^+$', r'$\sum_{q^+ = u,d,s,c}$', r'$g$', 'Total']
labels = [r'$u$', r'$d$', r'$s$', r'$c$', 'Total']
colors = ['r','g','b','orange','grey']

ax=axs[0,0]
ax.axhline(1/2,color='black',ls='--',marker='')
ax.axhline(0,color='black',ls='--',marker='')
ax.set_ylabel(r'$\frac{{1}}{{2}}\Delta\Sigma_{q}$')

x = np.arange(len(labels))
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_xlim([x[0]-3/4,x[-1]+3/4])

mes_conn=[yu.jackme(get(j,stout,'conn')) for j in js]
mes=[yu.jackme(get(j,stout,'full')) for j in js]

factor=0.5
    
ax.bar(x, [ele[0] for ele in mes_conn], capsize=5, color=colors, alpha=0.8, width=0.3, edgecolor='grey')
bars=ax.bar(x, [ele[0] for ele in mes], yerr=[ele[1] for ele in mes], capsize=5, color=colors, alpha=0.2, width=0.5, edgecolor='grey')

func=lambda res:'{:.1f}({:.1f})%'.format(res[0]*100/factor,res[1]*100/factor)
percentages=[func(ele) for ele in mes]

if not noNumbersOnBar:
    for i, (bar, pct) in enumerate(zip(bars, percentages)):
        height = bar.get_height()
        height = 0.01 if height < 0.1 else height - 0.1
        
        if bar.get_height()<0.01:
            height *= -1
        
        ax.text(bar.get_x() - 0.1, height, pct,
                ha='center', va='bottom' if bar.get_height()>0.01 else 'top', fontsize=10, rotation=90)
        
plt.tight_layout()
plt.savefig(f'{baseFigPath}/final1/percentages_DeltaSigmaBy2.pdf',bbox_inches="tight")
plt.close()

# L

In [29]:
def get(ens,j):
    return (ens2rb_A20[ens][j]+ens2rb_B20[ens][j])/2 - ens2rb_gA[ens][j]/2

stout=stout_chosen

j2color={'jq':'grey','jg':'cyan','jtot':'grey','ju':'r','jd':'g','js':'b','jc':'orange'}
j2label={'jq':'N','jg':'g','jtot':'N','ju':'u','jd':'d','js':'s','jc':'c'}
j2fmt={'jq':'d','jg':'s','jtot':'o','ju':'^','jd':'v','js':'<','jc':'>'}

fig, axs = yu.getFigAxs(1,1,Lrow=4,Lcol=6,sharex=True)
ax=axs[0,0]
ax.set_xlim([0,0.01])
ax.set_xticks([0,0.003,0.006,0.009])

ax.set_xlabel(r'$a^2$ [fm$^2$]')
ax.set_ylim([-0.3,0.5])
ax.set_yticks([-0.2,0,0.2,0.4])
ax.set_ylabel(rf'$L_q$')
# ax.axhline(0,color='black',ls='--',marker='')
# ax.axhline(0.5,color='black',ls='--',marker='')
js=['ju','jd','js','jc','jq']
for ij,j in enumerate(js):
    color=j2color[j]
    mean,err=yu.jackme(get('a=#_MA',f'{j};{stout}'))
    label=rf'$L_{{{j2label[j]}}}=$'+yu.un2str(mean[0],err[0],forceResult=1)
    if noNumbersOnLegend:
        label=rf'$L_{{{j2label[j]}}}$'
    for iens,ens in enumerate(enss):
        plt_x=ens2a[ens]**2+(ij-len(js)/2)*5e-5; plt_y,plt_yerr=yu.jackme(get(ens,f'{j};{stout}'))
        ax.errorbar(plt_x,plt_y,plt_yerr,color=color,fmt=j2fmt[j],label=label if iens==0 else None)

    mean,err=yu.jackme(get('a=#_MA',f'{j};{stout}'))
    x=a2s_plt; ymin=mean-err; ymax=mean+err
    ax.plot(x,mean,color=color,linestyle='--',marker='')
    ax.fill_between(x, ymin, ymax, color=color, alpha=0.1)
ax.legend(fontsize=12,ncol=3)

plt.tight_layout()
plt.savefig(f'{baseFigPath}/final1/a2Dependence_L.pdf',bbox_inches="tight")
plt.close()

In [64]:
stout=stout_chosen

def get0(ens,j):
    return (ens2rb_A20[ens][j]+ens2rb_B20[ens][j])/2 - ens2rb_gA[ens][j]/2

def get(j,stout,part):
    if part=='full':
        return get0('a=#_MA',f'{j};{stout}')[:,0]
    if part=='conn':
        if j not in ['ju','jd','jq','jtot']:
            return get0('a=#_MA',f'{j};{stout}')[:,0]*0
        return get0('a=#_MA',f'{j};conn;{stout}')[:,0]

fig, axs = yu.getFigAxs(1,1,Lrow=6,Lcol=8)

js = ['ju','jd','js','jc','jq']
# labels = [r'$u^+$', r'$d^+$', r'$s^+$', r'$c^+$', r'$\sum_{q^+ = u,d,s,c}$', r'$g$', 'Total']
labels = [r'$u$', r'$d$', r'$s$', r'$c$', 'Total']
colors = ['r','g','b','orange','grey']

ax=axs[0,0]
ax.axhline(1/2,color='black',ls='--',marker='')
ax.axhline(0,color='black',ls='--',marker='')
ax.set_ylabel(r'$L_{q}$')

x = np.arange(len(labels))
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_xlim([x[0]-3/4,x[-1]+3/4])

mes_conn=[yu.jackme(get(j,stout,'conn')) for j in js]
mes=[yu.jackme(get(j,stout,'full')) for j in js]

factor=0.5
    
ax.bar(x, [ele[0] for ele in mes_conn], capsize=5, color=colors, alpha=0.8, width=0.3, edgecolor='grey')
bars=ax.bar(x, [ele[0] for ele in mes], yerr=[ele[1] for ele in mes], capsize=5, color=colors, alpha=0.2, width=0.5, edgecolor='grey')

func=lambda res:'{:.1f}({:.1f})%'.format(res[0]*100/factor,res[1]*100/factor)
percentages=[func(ele) for ele in mes]

if not noNumbersOnBar:
    for i, (bar, pct) in enumerate(zip(bars, percentages)):
        height = bar.get_height()
        height = 0.01 if height < 0.2 else height - 0.1
        if bar.get_height()<-0.01:
            height = bar.get_height() + 0.1
        ax.text(bar.get_x() - 0.1, height, pct,
                ha='center', va='bottom' if bar.get_height()>-0.01 else 'top', fontsize=10, rotation=90)
        
plt.tight_layout()
plt.savefig(f'{baseFigPath}/final1/percentages_L.pdf',bbox_inches="tight")
plt.close()

# mass decomposition

In [94]:
def get(j):
    dic={
        'Mm':yu.jackknife_pseudo(0.165,0.016,100)[:,0]*100,
        'Mq':yu.jackknife_pseudo(0.215,0.038,100)[:,0]*100,
        'Mg':yu.jackknife_pseudo(0.358,0.022,100)[:,0]*100,
        'MN':yu.jackknife_pseudo(0.936,0.006,100)[:,0]*100,
    }
    
    dic['Ma']=(dic['MN']-dic['Mm'])/4
    dic['M']=np.sum([dic[j] for j in ['Mm','Ma','Mq','Mg']],axis=0)

    return dic[j]/dic['MN']*100

fig, axs = yu.getFigAxs(1,1,Lrow=6,Lcol=8)

js = ['Mm','Ma','Mq','Mg','M']
# labels = [r'$u^+$', r'$d^+$', r'$s^+$', r'$c^+$', r'$\sum_{q^+ = u,d,s,c}$', r'$g$', 'Total']
labels = [r'$M_m$', r'$M_a$', r'$M_q$', r'$M_g$', r'$M$']
colors = ['b','orange','green','red','purple']

ax=axs[0,0]
ax.axhline(100,color='black',ls='--',marker='')
# ax.axhline(0,color='black',ls='--',marker='')
ax.set_ylabel(r'% of M=936(6) MeV')

x = np.arange(len(labels))
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_xlim([x[0]-3/4,x[-1]+3/4])

mes=[yu.jackme(get(j)) for j in js]

factor=100
    
bars=ax.bar(x, [ele[0] for ele in mes], yerr=[ele[1] for ele in mes], capsize=5, color=colors, alpha=0.8, width=0.5, edgecolor='grey')

func=lambda res:'{:.1f}({:.1f})%'.format(res[0]*100/factor,res[1]*100/factor)
percentages=[func(ele) for ele in mes]

if not noNumbersOnBar:
    for i, (bar, pct) in enumerate(zip(bars, percentages)):
        height = bar.get_height()
        height = 1 if height < 20 else height - 20
        ax.text(bar.get_x() + 0.6, height, pct,
                ha='center', va='bottom', fontsize=10, rotation=90)
        
plt.tight_layout()
plt.savefig(f'{baseFigPath}/final1/percentages_m_sumrule1.pdf',bbox_inches="tight")
plt.close()

In [93]:
def get(j):
    dic={
        'Mm':yu.jackknife_pseudo(0.165,0.016,100)[:,0]*100,
        'Mq':yu.jackknife_pseudo(0.215,0.038,100)[:,0]*100,
        'Mg':yu.jackknife_pseudo(0.358,0.022,100)[:,0]*100,
        'MN':yu.jackknife_pseudo(0.936,0.006,100)[:,0]*100,
    }
    
    dic['Ma']=dic['MN']-dic['Mm']-dic['Mq']-dic['Mg']
    dic['M']=np.sum([dic[j] for j in ['Mm','Ma','Mq','Mg']],axis=0)

    return dic[j]/dic['MN']*100

fig, axs = yu.getFigAxs(1,1,Lrow=6,Lcol=8)

js = ['Mm','Ma','Mq','Mg','M']
# labels = [r'$u^+$', r'$d^+$', r'$s^+$', r'$c^+$', r'$\sum_{q^+ = u,d,s,c}$', r'$g$', 'Total']
labels = [r'$M_m$', r'$M_a$', r'$M_q$', r'$M_g$', r'$M$']
colors = ['b','orange','green','red','purple']

ax=axs[0,0]
ax.axhline(100,color='black',ls='--',marker='')
# ax.axhline(0,color='black',ls='--',marker='')
ax.set_ylabel(r'% of M=936(6) MeV')

x = np.arange(len(labels))
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_xlim([x[0]-3/4,x[-1]+3/4])

mes=[yu.jackme(get(j)) for j in js]

factor=100
    
bars=ax.bar(x, [ele[0] for ele in mes], yerr=[ele[1] for ele in mes], capsize=5, color=colors, alpha=0.8, width=0.5, edgecolor='grey')

func=lambda res:'{:.1f}({:.1f})%'.format(res[0]*100/factor,res[1]*100/factor)
percentages=[func(ele) for ele in mes]

if not noNumbersOnBar:
    for i, (bar, pct) in enumerate(zip(bars, percentages)):
        height = bar.get_height()
        height = 1 if height < 20 else height - 20
        ax.text(bar.get_x() + 0.6, height, pct,
                ha='center', va='bottom', fontsize=10, rotation=90)
        
plt.tight_layout()
plt.savefig(f'{baseFigPath}/final1/percentages_m_sumrule2.pdf',bbox_inches="tight")
plt.close()